In [ ]:
# Add the relevant scripts from LArMachineLearningData
# Nice the process so it can run with lots of cores on low priority
import os
os.nice(20)

# Add local paths
import sys
hnlDIR = os.environ['_']
sys.path.append('../pyscript')

# From pyscript Library
from Plotting import *
from Dictionary import *
from HelperFunctions import *
from CutFunctions import *

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

<h1>Specify Mass</h1>

In [ ]:
m = 240

<h1>Configuration Stuff Here</h1>

In [ ]:
ifSave = True
savePath = "../../plot_files/14April2024_m"+str(m)+"_v3_selection/"

In [ ]:
ifCut = True

In [ ]:
ifTune = False

#Common cut value
TuneCut = {
    'beamSpillBegin': 367 - 16
    , 'beamSpillEnd': 367 + 1600 + 16
    , 'crumbs': 0 
    , 'Muon' : 0.04
    , 'Proton' : 0.96
    , 'Pion' : 0.82 
    , 'Electron' : 0.98 
    #, 'Photon' : 0.02 
    , 'ShowerEnergy': 50 
    #, 'PE': 2500
    , 'OneShowerFracMore': -0.12
    , 'OneShowerFracLess': 0.4
    , 'TwoShowerFracMore': 0
    , 'TwoShowerFracLess': 0.4    
    , 'OneShowerTheta': 20
    , 'TwoShowerTheta': 30 
    }


<h1> Read in PKL Dataframe </h1>

<h3> HNL </h3>

In [ ]:
hnl_path ="../../pkl_files/v3_April2024/df_preselect_m"+str(m)+"_v3_hnl.pkl"
print(hnl_path)
    
file_hnl = open(hnl_path, 'rb')
df_hnl = pickle.load(file_hnl)

del hnl_path
file_hnl.close()

<h3> Neutrino </h3>

In [ ]:
nu_path ="../../pkl_files/v3_April2024/df_preselect_v3_nu.pkl"
print(nu_path)
    
file_nu = open(nu_path, 'rb')
df_nu = pickle.load(file_nu)

del nu_path
file_nu.close()

<h3> Intime Cosmics </h3>

In [ ]:
cos_path ="../../pkl_files/v3_April2024/df_preselect_v3_cos.pkl"
print(cos_path)
    
file_cos = open(cos_path, 'rb')
df_cos = pickle.load(file_cos)

del cos_path
file_cos.close()

<h1>Combine FV and Non-FV HNL</h1>

In [ ]:
df_hnl['slc_true_event_type'][df_hnl['slc_true_event_type'] == 1] = 0
df_nu['slc_true_event_type'][df_nu['slc_true_event_type'] == 1] = 0
df_cos['slc_true_event_type'][df_cos['slc_true_event_type'] == 1] = 0

<h1>Scale HNL for Plotting</h1>

In [ ]:
simU = df_hnl['simU'][0]
true_counts = df_hnl['true_counts'][0]
start_counts = df_hnl['start_counts'][0]


scaleHNLPlot = 1
    
if m ==200:    
    scaleHNLPlot = 100
else:
    scaleHNLPlot = 148118.81420359976 / true_counts

print("Sim Umu = " +str(simU))
print("true counts = " + str(true_counts))
print("start counts = " + str(start_counts))    
print("Scale factor " + str(scaleHNLPlot))

In [ ]:
plotU = getUfromScaleFactor(simU, scaleHNLPlot)

true_counts = true_counts * scaleHNLPlot
start_counts =  start_counts * scaleHNLPlot


print("Scaled Umu = " + str(plotU))
print("Scaled true counts = " + str(true_counts))
print("Scaled start counts = " + str(start_counts))

In [ ]:
df_hnl['sim_scale_pot'] = df_hnl['scale_pot']
df_hnl['scale_pot'] = df_hnl['sim_scale_pot'] * scaleHNLPlot
df_hnl['plotU'] = plotU

In [ ]:
label = str(m) + ' MeV HNL ${\pi}^{0}$' + '\n' + '|U$_{{\mu 4}}$|$^{{2}}$ = ' +str(sci_notation(plotU,2,2))
print(label)

<h1> Make Beam Bucket - Post Non-clear cosmics </h1>

In [ ]:
width = 18.936

df_hnl['mod_t'] = df_hnl['slc_opt0_time_corrected_Z_pandora']%width
df_nu['mod_t'] = df_nu['slc_opt0_time_corrected_Z_pandora']%width
df_cos['mod_t'] = df_cos['slc_opt0_time_corrected_Z_pandora']%width
#----------------------------------------------
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    xmin = 0, xmax = 30, xnbin = 30,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',
                    ifAddLegend = True, addLegend = label
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_post_nonclearcosmics.png", dpi = 200)
#-----------------------------------#
plt.show()

<h3> Save Purity and Eff </h3>

In [ ]:
purity_arr, eff_arr, cut_arr = [], [], []

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)


purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Clear Cosmics Cut")

print(purity_arr)
print(eff_arr)
print(cut_arr)

<h1> Beam Spill </h1>

In [ ]:
light_speed_cm_us = 29979.2458 # cm/ns

df_hnl['slc_opt0_time'] = df_hnl['slc_opt0_time_corrected_Z_pandora'] + df_hnl['slc_vtx_z']/light_speed_cm_us
df_nu['slc_opt0_time'] = df_nu['slc_opt0_time_corrected_Z_pandora'] + df_nu['slc_vtx_z']/light_speed_cm_us
df_cos['slc_opt0_time'] = df_cos['slc_opt0_time_corrected_Z_pandora'] + df_cos['slc_vtx_z']/light_speed_cm_us
#-----------------------------------#
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_opt0_time', 
                    xmin = 0, xmax = 3200, xnbin = 100,
                    xtitle = 'Opt0 Time [ns]',
                    ifAddLegend = True, addLegend = label
                    )

plt.axvline(x = TuneCut['beamSpillBegin'], c = 'r', linestyle = '--', lw = 1.5)
plt.axvline(x = TuneCut['beamSpillEnd'], c = 'r', linestyle = '--', lw = 1.5)

plt.ylim(0, 1.25e6)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beamspill.png", dpi = 200)
#-----------------------------------#
plt.show()

<h3>Apply Cut</h3>

In [ ]:
if ifCut:
    df_hnl = cutBeamGate(df_hnl, gateBegin = TuneCut['beamSpillBegin'], gateEnd = TuneCut['beamSpillEnd'])
    df_nu = cutBeamGate(df_nu, gateBegin = TuneCut['beamSpillBegin'], gateEnd = TuneCut['beamSpillEnd'])
    df_cos = cutBeamGate(df_cos, gateBegin = TuneCut['beamSpillBegin'], gateEnd = TuneCut['beamSpillEnd'])

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Beam Spill")

print(purity_arr)
print(eff_arr)
print(cut_arr)

<h1> Make Beam Bucket - Post Beam Spill</h1>

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    xmin = 0, xmax = 30, xnbin = 30,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',
                    ifAddLegend = True, addLegend = label
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_post_beamspill.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1> CRUMBS </h1>

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_crumbs_score', 
                    xmin = -2, xmax =1, xnbin = 30,
                    xtitle = 'CRUMBS Score',
                    ifAddLegend = True, addLegend = label
                    )

plt.axvline(x = TuneCut['crumbs'], c = 'r', linestyle = '--', lw = 1.5)

plt.annotate("", xy=(0, 4.5e6), xytext=(0.4, 4.5e6)
             , arrowprops=dict(arrowstyle= '<|-',
                             color='r',
                             lw=1.5,
                             ls='--'))

plt.ylim(0, 5e6)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"crumbs_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h3>Apply Cut</h3>

In [ ]:
score = TuneCut['crumbs']

print(score)

if ifCut:
    df_hnl = cutCosmics(df_hnl, crumbsScore = score)
    df_nu = cutCosmics(df_nu, crumbsScore = score)
    df_cos = cutCosmics(df_cos, crumbsScore = score)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("CRUMBS")

print(purity_arr)
print(eff_arr)
print(cut_arr)

<h1> Make Beam Bucket - Post Crumbs</h1>

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    xmin = 0, xmax = 30, xnbin = 30,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + " [ns]",
                    ifAddLegend = True, addLegend = label
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_post_crumbs.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1> Fiducial Volume </h1>

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_is_fv', 
                    xmin = -0.5, xmax = 2.5, xnbin = 40,
                    xtitle = 'Vertex is in Fiducial Volume',
                     ifAddLegend = True, addLegend = label
                    )

plt.xticks(range(0, 2), labels=["False", "True"])

plt.axvline(x = 0.5, c = 'r', linestyle = '--', lw = 1.5)

plt.annotate("", xy=(0.5, 1.6e7), xytext=(0.9, 1.6e7)
             , arrowprops=dict(arrowstyle= '<|-',
                             color='r',
                             lw=1.5,
                             ls='--'))
plt.ylim(0, 1.8e7)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"fv_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h3>Apply Cut</h3>

In [ ]:
if ifCut:
    df_hnl = cutFV(df_hnl)
    df_nu = cutFV(df_nu)
    df_cos = cutFV(df_cos)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Vertex In FV")

print(purity_arr)
print(eff_arr)
print(cut_arr)

<h1> Make Beam Bucket - Post FV</h1>

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    xmin = 0, xmax = 30, xnbin = 30,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + " [ns]",
                     ifAddLegend = True, addLegend = label
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_post_FV.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1> Containment</h1>

<h3>Apply Cut</h3>

<h3>Save Purity/Eff</h3>

<h1> Make Beam Bucket - Post Containment</h1>

<h1>Muon Rejection</h1>

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_n_razzled_muons', 
                    xmin = 0, xmax = 6, xnbin = 6,
                    xtitle = 'n Razzled Muons',
                     ifAddLegend = True, addLegend = label
                    )

plt.axvline(x = 1, c = 'r', linestyle = '--', lw = 1.5)
plt.annotate("", xy=(0.5, 8.5e6), xytext=(1, 8.5e6)
             , arrowprops=dict(arrowstyle= '-|>',
                             color='r',
                             lw=1.5,
                             ls='--'))
plt.ylim(0, 9e6)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nrazzled_muon_precut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_razzled_muon_score', 
                    xmin = 0, xmax = 1, xnbin = 50,
                    xtitle = 'PFP Razzled Muon Score',
                     ifAddLegend = True, addLegend = label
                    )
plt.axvline(x =  TuneCut['Muon'], c = 'r', linestyle = '--', lw = 1.5)

plt.annotate("", xy=(0.0, 8.5e6), xytext=(TuneCut['Muon'], 8.5e6)
             , arrowprops=dict(arrowstyle= '-|>',
                             color='r',
                             lw=1.5,
                             ls='--'))
plt.ylim(0, 9e6)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_muon_score_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h3> Apply Cut</h3>

In [ ]:
muonScore = TuneCut['Muon']

print(muonScore)

if ifCut:
    df_hnl = cutMuon(df_hnl, muonScore =muonScore)
    df_nu = cutMuon(df_nu, muonScore =muonScore)
    df_cos = cutMuon(df_cos,muonScore =muonScore)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Muon Cut")

print(purity_arr)
print(eff_arr)
print(cut_arr)

<h1> Make Beam Bucket - Post Muon</h1>

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    xmin = 0, xmax = 30, xnbin = 30,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + " [ns]",
                     ifAddLegend = True, addLegend = label
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_postcosmics_postmuon.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Proton Rejection</h1>

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_n_razzled_protons_thresh', 
                    xmin = 0, xmax = 6, xnbin = 6,
                    xtitle = 'n Razzled Protons',
                     ifAddLegend = True, addLegend = label
                    )

plt.axvline(x = 1, c = 'r', linestyle = '--', lw = 1.5)
plt.annotate("", xy=(0.5, 9.5e5), xytext=(1, 9.5e5)
             , arrowprops=dict(arrowstyle= '-|>',
                             color='r',
                             lw=1.5,
                             ls='--'))
plt.ylim(0, 1e6)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nrazzled_proton_precut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_razzled_proton_score', 
                    xmin = 0, xmax = 1, xnbin = 50,
                    xtitle = 'PFP Razzled Proton Score',
                      ifAddLegend = True, addLegend = label, LegendLoc = 'upper center'
                    )

plt.axvline(x =  TuneCut['Proton'], c = 'r', linestyle = '--', lw = 1.5)

plt.annotate("", xy=(TuneCut['Proton'], 7.5e5), xytext=(TuneCut['Proton'] - 0.1, 7.5e5)
             , arrowprops=dict(arrowstyle= '<|-',
                             color='r',
                             lw=1.5,
                             ls='--'))
plt.ylim(0, 8e5)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_proton_score_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h3>Apply Cut</h3>

In [ ]:
protonScore = TuneCut['Proton']

print(protonScore)

if ifCut:
    df_hnl = cutProton(df_hnl, nProton = 0, protonScore = protonScore)
    df_nu = cutProton(df_nu, nProton = 0, protonScore = protonScore)
    df_cos = cutProton(df_cos, nProton = 0, protonScore = protonScore)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Proton Cut")

print(purity_arr)
print(eff_arr)
print(cut_arr)

<h1> Make Beam Bucket - Post Proton</h1>

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    xmin = 0, xmax = 30, xnbin = 30,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + " [ns]",
                 ifAddLegend = True, addLegend = label
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_postcosmics_postproton.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Pion Rejection</h1>

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_n_razzled_pions_thresh', 
                    xmin = 0, xmax = 6, xnbin = 6,
                    xtitle = 'n Razzled Pions',
                      ifAddLegend = True, addLegend = label
                    )

plt.axvline(x = 1, c = 'r', linestyle = '--', lw = 1.5)
plt.annotate("", xy=(0.5, 6.5e5), xytext=(1, 6.5e5)
             , arrowprops=dict(arrowstyle= '-|>',
                             color='r',
                             lw=1.5,
                             ls='--'))
plt.ylim(0, 7e5)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nrazzled_pion_precut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_razzled_pion_score', 
                    xmin = 0, xmax = 1, xnbin = 50,
                    xtitle = 'PFP Razzled Pion Score',
                      ifAddLegend = True, addLegend = label, LegendLoc = 'upper center'
                    )

plt.axvline(x =  TuneCut['Pion'], c = 'r', linestyle = '--', lw = 1.5)

plt.annotate("", xy=(TuneCut['Pion'], 2.5e5), xytext=(TuneCut['Pion'] - 0.1, 2.5e5)
             , arrowprops=dict(arrowstyle= '<|-',
                             color='r',
                             lw=1.5,
                             ls='--'))
plt.ylim(0, 3e5)

#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_pion_score_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h3>Apply Cut </h3>

In [ ]:
pionScore = TuneCut['Pion']

print(pionScore)

if ifCut:
    df_hnl = cutPion(df_hnl, nPion = 0, pionScore = pionScore)
    df_nu = cutPion(df_nu, nPion = 0, pionScore = pionScore)
    df_cos = cutPion(df_cos, nPion = 0, pionScore = pionScore)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Pion Cut")

print(purity_arr)
print(eff_arr)
print(cut_arr)

<h1> Make Beam Bucket - Post Pion</h1>

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    xmin = 0, xmax = 30, xnbin = 30,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + " [ns]",
                     ifAddLegend = True, addLegend = label
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_postcosmics_postpion.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Electron Rejection</h1>

<h3> Get Primary Shower </h3>

In [ ]:
df_hnl_prim = get_primary_shw_df(df_hnl)

df_nu_prim = get_primary_shw_df(df_nu)

df_cos_prim = get_primary_shw_df(df_cos)

<h3> Primary Shower </h3>

In [ ]:
#----------------------------------------------
plot_slc_var(df_hnl_prim[df_hnl_prim['slc_pfp_razzled_electron_score'] < 2]
            , df_nu_prim[df_nu_prim['slc_pfp_razzled_electron_score'] < 2]
             , df_cos_prim[df_cos_prim['slc_pfp_razzled_electron_score'] < 2],
                    true_counts, start_counts, 
                    'slc_pfp_razzled_electron_score', 
                    xmin = 0, xmax = 1, xnbin = 50,
                    xtitle = 'PFP Razzled Electron Score',
                  ifAddLegend = True, addLegend = label, LegendLoc ='upper center'
                    )

plt.axvline(x =  TuneCut['Electron'], c = 'r', linestyle = '--', lw = 1.5)

plt.annotate("", xy=(TuneCut['Electron'], 3.5e4), xytext=(TuneCut['Electron'] - 0.1, 3.5e4)
             , arrowprops=dict(arrowstyle= '<|-',
                             color='r',
                             lw=1.5,
                             ls='--'))
plt.ylim(0, 4e4)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_electron_score_prim_shw_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h3> Apply Cut </h3>

In [ ]:
eScore = TuneCut['Electron']

print(eScore)

if ifCut:
    df_hnl_prim = cutElectronLess(df_hnl_prim, ifnElectron = False, electronScore = eScore)
    df_nu_prim = cutElectronLess(df_nu_prim, ifnElectron = False, electronScore = eScore)
    df_cos_prim = cutElectronLess(df_cos_prim, ifnElectron = False, electronScore = eScore)

In [ ]:
df_hnl = merge_df_prim(df_hnl, df_hnl_prim)

df_nu = merge_df_prim(df_nu, df_nu_prim)

df_cos = merge_df_prim(df_cos, df_cos_prim)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Electron Cut")

print(purity_arr)
print(eff_arr)
print(cut_arr)

<h1> Make Beam Bucket - Post Electron</h1>

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    xmin = 0, xmax = 30, xnbin = 30,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + " [ns]",
                     ifAddLegend = True, addLegend = label
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_postcosmics_postelectron.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Photon Selection (?)</h1>

<h3> Get Primary Shower </h3>

<h3> Apply Cut </h3>

<h3>Save Purity/Eff</h3>

<h1> Make Beam Bucket - Post Photon</h1>

<h1>Opt0 Measured PE </h1>

<h3>Both 1 and 2+ Shower in Slice</h3>

<h3> Split 1 and 2+ shower cases </h3>

In [ ]:
df_hnl_shw1, df_hnl_shw2 = split_my_df(df_hnl)

df_nu_shw1, df_nu_shw2 = split_my_df(df_nu)

df_cos_shw1, df_cos_shw2 = split_my_df(df_cos)

<h3> 1 shower cases </h3>

In [ ]:
#plot_slc_var(df_hnl_shw1, df_nu_shw1, df_cos,
plot_slc_var(df_hnl_shw1, df_nu_shw1, df_cos_shw1,
                    true_counts, start_counts, 
                    'slc_opt0_measPE', 
                    xmin = 0, xmax = 100000, xnbin = 40,
                    xtitle = 'Opt0 Measured PE',
                      ifAddLegend = True, addLegend = label
                    )

#plt.axvline(x =  TuneCut['PE'], c = 'r', linestyle = '--', lw = 1.5)

#plt.annotate("", xy=(TuneCut['PE'], 1e4), xytext=(TuneCut['PE'] + 7500, 1e4)
#             , arrowprops=dict(arrowstyle= '<|-',
#                             color='r',
#                             lw=1.5,
#                             ls='--'))
#plt.ylim(0, 1.5e4)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"one_shower_opt0PE_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h3> 2+ shower cases </h3>

In [ ]:
#plot_slc_var(df_hnl_shw2, df_nu_shw2, df_cos,
plot_slc_var(df_hnl_shw2, df_nu_shw2, df_cos_shw2,
                    true_counts, start_counts, 
                    'slc_opt0_measPE', 
                    xmin = 0, xmax = 100000, xnbin = 40,
                    xtitle = 'Opt0 Measured PE',
                     ifAddLegend = True, addLegend = label
                    )

#plt.axvline(x =  TuneCut['PE'], c = 'r', linestyle = '--', lw = 1.5)

#plt.annotate("", xy=(TuneCut['PE'], 2.5e4), xytext=(TuneCut['PE'] + 7500, 2.5e4)
#             , arrowprops=dict(arrowstyle= '<|-',
#                             color='r',
#                             lw=1.5,
#                             ls='--'))
#plt.ylim(0, 3e4)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"one_shower_opt0PE_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h3>Apply Cut</h3>

<h3>Save Purity/Eff</h3>

<h1> Make Beam Bucket - Post PE</h1>

<h1>Shower Energy, dEdx and Conversion Gap</h1>

<h3> Get Primary Shower </h3>

In [ ]:
df_hnl_prim = get_primary_shw_df(df_hnl)

df_nu_prim = get_primary_shw_df(df_nu)

df_cos_prim = get_primary_shw_df(df_cos)

<h3> Primary Showers of both 1 and 2+ shower cases </h3>

<h3> Split into 1 shower and multi shower case </h3>

In [ ]:
df_hnl_shw1, df_hnl_shw2 = split_my_df(df_hnl)

df_nu_shw1, df_nu_shw2 = split_my_df(df_nu)

df_cos_shw1, df_cos_shw2 = split_my_df(df_cos)

<h3> Primary Showers of 1 shower cases </h3>

In [ ]:
plot_slc_var(df_hnl_shw1, df_nu_shw1, df_cos_shw1,
#plot_slc_var(df_hnl_shw1, df_nu_shw1, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_energy', 
                    xmin = 0, xmax = 2000, xnbin = 40,
                    xtitle = 'Shower Energy [MeV]',
                      ifAddLegend = True, addLegend = label
                         )

plt.axvline(x =  TuneCut['ShowerEnergy'], c = 'r', linestyle = '--', lw = 1.5)

plt.annotate("", xy=(TuneCut['ShowerEnergy'], 1.1e4), xytext=(TuneCut['ShowerEnergy'] + 100,  1.1e4)
             , arrowprops=dict(arrowstyle= '<|-',
                             color='r',
                             lw=1.5,
                             ls='--'))
plt.ylim(0, 1.2e4)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"one_shower_energy_postpid.png", dpi = 200)
#-----------------------------------#
plt.show()
#================================================#
plot_slc_var(df_hnl_shw1, df_nu_shw1, df_cos_shw1,
#plot_slc_var(df_hnl_shw1, df_nu_shw1, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_dedx', 
                    xmin = 0, xmax = 15, xnbin = 40,
                    xtitle = 'Shower dE/dx [MeV]',
                      ifAddLegend = True, addLegend = label
                         )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"one_shower_dedx_postpid.png", dpi = 200)
#-----------------------------------#
plt.show()    
    
#================================================#
plot_slc_var(df_hnl_shw1, df_nu_shw1, df_cos_shw1,
#plot_slc_var(df_hnl_shw1, df_nu_shw1, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_conv_gap', 
                    xmin = 0, xmax = 5, xnbin = 40,
                    xtitle = 'Shower Conversion Gap [cm]',
                      ifAddLegend = True, addLegend = label
                         )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"one_shower_convgap_postpid.png", dpi = 200)
#-----------------------------------#
plt.show()

<h3> Primary Showers of 2+ shower cases </h3>

In [ ]:
df_nu_shw2 = df_nu_shw2.apply(pd.Series.explode).reset_index()
df_hnl_shw2 = df_hnl_shw2.apply(pd.Series.explode).reset_index()
df_cos_shw2 = df_cos_shw2.apply(pd.Series.explode).reset_index()

In [ ]:
df_hnl_prim_shw2 = get_primary_shw_df(df_hnl_shw2)

df_nu_prim_shw2 = get_primary_shw_df(df_nu_shw2)

df_cos_prim_shw2 = get_primary_shw_df(df_cos_shw2)
#df_cos_prim_shw2 = get_primary_shw_df(df_cos)

In [ ]:
plot_slc_var(df_hnl_prim_shw2, df_nu_prim_shw2, df_cos_prim_shw2,
                    true_counts, start_counts, 
                    'slc_pfp_shower_energy', 
                    xmin = 0, xmax = 2000, xnbin = 40,
                    xtitle = 'Shower Energy [MeV]',
                      ifAddLegend = True, addLegend = label
                         )

plt.axvline(x =  TuneCut['ShowerEnergy'], c = 'r', linestyle = '--', lw = 1.5)

plt.annotate("", xy=(TuneCut['ShowerEnergy'], 3e4), xytext=(TuneCut['ShowerEnergy'] + 100,  3e4)
             , arrowprops=dict(arrowstyle= '<|-',
                             color='r',
                             lw=1.5,
                             ls='--'))
plt.ylim(0, 3.1e4)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"two_prim_shower_energy_postpid.png", dpi = 200)
#-----------------------------------#
plt.show()

#================================================#
plot_slc_var(df_hnl_prim_shw2, df_nu_prim_shw2, df_cos_prim_shw2,
                    true_counts, start_counts, 
                    'slc_pfp_shower_dedx', 
                    xmin = 0, xmax = 15, xnbin = 40,
                    xtitle = 'Shower dE/dx [MeV]',
                      ifAddLegend = True, addLegend = label
                         )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"two_prim_shower_dedx_postpid.png", dpi = 200)
#-----------------------------------#
plt.show()    
    
#================================================#
plot_slc_var(df_hnl_prim_shw2, df_nu_prim_shw2, df_cos_prim_shw2,
                    true_counts, start_counts, 
                    'slc_pfp_shower_conv_gap', 
                    xmin = 0, xmax = 5, xnbin = 40,
                    xtitle = 'Shower Conversion Gap [cm]',
                      ifAddLegend = True, addLegend = label
                         )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"two_prim_shower_convgap_postpid.png", dpi = 200)
#-----------------------------------#
plt.show()

<h3>Apply Cut</h3>

In [ ]:
energy = TuneCut['ShowerEnergy']

print(energy)

if ifCut:
    df_hnl_prim = df_hnl_prim[df_hnl_prim['slc_pfp_shower_energy'] >= energy]
    df_nu_prim = df_nu_prim[df_nu_prim['slc_pfp_shower_energy'] >= energy]
    df_cos_prim = df_cos_prim[df_cos_prim['slc_pfp_shower_energy'] >= energy]

In [ ]:
df_hnl = merge_df_prim(df_hnl, df_hnl_prim)

df_nu = merge_df_prim(df_nu, df_nu_prim)

df_cos = merge_df_prim(df_cos, df_cos_prim)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Minimum Energy")

print(purity_arr)
print(eff_arr)
print(cut_arr)

<h1> Make Beam Bucket - Post Shower Energy</h1>

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    xmin = 0, xmax = 30, xnbin = 30,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',
                    ifAddLegend = True, addLegend = label
                    )

#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_postShwE.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Opt0 Frac </h1>

<h3>Both 1 and 2+ Shower in Slice</h3>

<h3> Split 1 and 2+ shower cases </h3>

In [ ]:
df_hnl_shw1, df_hnl_shw2 = split_my_df(df_hnl)

df_nu_shw1, df_nu_shw2 = split_my_df(df_nu)

df_cos_shw1, df_cos_shw2 = split_my_df(df_cos)

<h3> 1 shower cases </h3>

In [ ]:
plot_slc_var(df_hnl_shw1, df_nu_shw1, df_cos_shw1,
                    true_counts, start_counts, 
                    'slc_opt0_frac', 
                    xmin = -1, xmax = 2, xnbin = 40,
                    xtitle = 'Opt0 Fraction',
                  ifAddLegend = True, addLegend = label
                    )
plt.axvline(x =  TuneCut['OneShowerFracMore'], c = 'r', linestyle = '--')
plt.axvline(x =  TuneCut['OneShowerFracLess'], c = 'r', linestyle = '--')
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"opt0frac_one_shw_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

print( TuneCut['OneShowerFracMore'])

In [ ]:
if ifTune:
    vary_OpT0FracMore(df_hnl_shw1, df_nu_shw1, df_cos_shw1, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/opt0frac_more_one_shw_tuning.png", dpi =200)

<h3> 2+ shower cases </h3>

In [ ]:
plot_slc_var(df_hnl_shw2, df_nu_shw2, df_cos_shw2,
                    true_counts, start_counts, 
                    'slc_opt0_frac', 
                    xmin = -1, xmax = 2, xnbin = 40,
                    xtitle = 'Opt0 Fraction',
                  ifAddLegend = True, addLegend = label
                    )
plt.axvline(x =  TuneCut['TwoShowerFracMore'], c = 'r', linestyle = '--')
plt.axvline(x =  TuneCut['TwoShowerFracLess'], c = 'r', linestyle = '--')
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"opt0frac_two_shw_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

print( TuneCut['TwoShowerFracMore'])

In [ ]:
if ifTune:
    vary_OpT0FracMore(df_hnl_shw2, df_nu_shw2, df_cos_shw2, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/opt0frac_more_two_shw_tuning.png", dpi =200)

<h3> Apply Cut </h3>

<h3> 1 shower only: harsh cut</h3>

In [ ]:
fracMore = TuneCut['OneShowerFracMore']
print(fracMore)
fracLess = TuneCut['OneShowerFracLess']
print(fracLess)

if ifCut:
    df_hnl_shw1 = cutOpt0Frac(df_hnl_shw1, Opt0FracMore = fracMore, Opt0FracLess = fracLess)
    df_nu_shw1 = cutOpt0Frac(df_nu_shw1, Opt0FracMore = fracMore, Opt0FracLess = fracLess)
    df_cos_shw1 = cutOpt0Frac(df_cos_shw1, Opt0FracMore = fracMore, Opt0FracLess = fracLess)

<h3> 2+ showers: cut on primary + less harsh cut </h3>

In [ ]:
#fracMore = TuneCut['Opt0FracMore']
fracMore = TuneCut['TwoShowerFracMore']
print(fracMore)
fracLess = TuneCut['TwoShowerFracLess']
print(fracLess)

if ifCut:
    df_hnl_shw2 = cutOpt0Frac(df_hnl_shw2, Opt0FracMore = fracMore, Opt0FracLess = fracLess)
    df_nu_shw2 = cutOpt0Frac(df_nu_shw2, Opt0FracMore = fracMore, Opt0FracLess = fracLess)
    df_cos_shw2 = cutOpt0Frac(df_cos_shw2, Opt0FracMore = fracMore, Opt0FracLess = fracLess)

In [ ]:
df_hnl = apply_shower_cut(df_hnl, df_hnl_shw1, df_hnl_shw2)

df_nu = apply_shower_cut(df_nu, df_nu_shw1, df_nu_shw2)

df_cos = apply_shower_cut(df_cos, df_cos_shw1, df_cos_shw2)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Opt0 Fraction Cut")

print(purity_arr)
print(eff_arr)
print(cut_arr)

<h1> Make Beam Bucket - Post Opt0 Frac</h1>

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    xmin = 0, xmax = 30, xnbin = 30,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',
                  ifAddLegend = True, addLegend = label
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_postopt0.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Shower Theta</h1>

<h3> Get Primary Shower </h3>

In [ ]:
df_hnl_prim = get_primary_shw_df(df_hnl)

df_nu_prim = get_primary_shw_df(df_nu)

df_cos_prim = get_primary_shw_df(df_cos)

<h3> Split 1 and 2+ shower cases </h3>

In [ ]:
df_hnl_shw1, df_hnl_shw2 = split_my_df(df_hnl)

df_nu_shw1, df_nu_shw2 = split_my_df(df_nu)

df_cos_shw1, df_cos_shw2 = split_my_df(df_cos)

<h3> Primary Showers of 1 shower cases </h3>

In [ ]:
plot_slc_var(df_hnl_shw1, df_nu_shw1, df_cos_shw1,
#plot_slc_var(df_hnl_shw1, df_nu_shw1, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_theta', 
                    xmin = 0, xmax = 180, xnbin = 36,
                    xtitle = 'Shower Theta',
                  ifAddLegend = True, addLegend = label
                    )

plt.axvline(x =  TuneCut['OneShowerTheta'], c = 'r', linestyle = '--', lw = 1.5)

plt.annotate("", xy=(TuneCut['OneShowerTheta'], 1.4e4), xytext=(TuneCut['OneShowerTheta'] - 15, 1.4e4)
             , arrowprops=dict(arrowstyle= '<|-',
                             color='r',
                             lw=1.5,
                             ls='--'))
plt.ylim(0, 1.5e4)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"one_shower_theta_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

print(TuneCut['OneShowerTheta'])

In [ ]:
if ifTune:
    vary_Theta(df_hnl_shw1, df_nu_shw1, df_cos_shw1, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/one_shower_theta_tuning.png", dpi =200)

<h3> Primary Showers of 2+ shower cases </h3>

In [ ]:
df_nu_shw2 = df_nu_shw2.apply(pd.Series.explode).reset_index()
df_hnl_shw2 = df_hnl_shw2.apply(pd.Series.explode).reset_index()
df_cos_shw2 = df_cos_shw2.apply(pd.Series.explode).reset_index()

In [ ]:
df_hnl_prim_shw2 = get_primary_shw_df(df_hnl_shw2)

df_nu_prim_shw2 = get_primary_shw_df(df_nu_shw2)

df_cos_prim_shw2 = get_primary_shw_df(df_cos_shw2)

In [ ]:
plot_slc_var(df_hnl_prim_shw2, df_nu_prim_shw2, df_cos_prim_shw2,
#plot_slc_var(df_hnl_shw2, df_nu_shw2, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_theta', 
                    xmin = 0, xmax = 180, xnbin = 36,
                    xtitle = 'Shower Theta',
                  ifAddLegend = True, addLegend = label
                    )


plt.axvline(x =  TuneCut['TwoShowerTheta'], c = 'r', linestyle = '--', lw = 1.5)

plt.annotate("", xy=(TuneCut['TwoShowerTheta'], 9.5e3), xytext=(TuneCut['TwoShowerTheta'] - 15, 9.5e3)
             , arrowprops=dict(arrowstyle= '<|-',
                             color='r',
                             lw=1.5,
                             ls='--'))
plt.ylim(0, 1e4)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"two_shower_primary_theta_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

print(TuneCut['TwoShowerTheta'])

In [ ]:
if ifTune:
    vary_Theta(df_hnl_prim_shw2, df_nu_prim_shw2, df_cos_prim_shw2, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/two_shower_prim_theta_tuning.png", dpi =200)

<h3>Apply Cut</h3>

<h3> 1 shower only: harsh cut</h3>

In [ ]:
angle = TuneCut['OneShowerTheta']
print(angle)

if ifCut:
    df_hnl_shw1 = cutThetaAngle(df_hnl_shw1, thetaAngle = angle) 
    df_nu_shw1 = cutThetaAngle(df_nu_shw1, thetaAngle = angle) 
    df_cos_shw1 = cutThetaAngle(df_cos_shw1, thetaAngle = angle) 
#df_cos = cutThetaAngle(df_cos, thetaAngle = angle) 

<h3> 2+ showers: cut on primary + less harsh cut </h3>

In [ ]:
angle = TuneCut['TwoShowerTheta']

print(angle)

if ifCut:
    df_hnl_prim_shw2 = cutThetaAngle(df_hnl_prim_shw2, thetaAngle = angle) 
    df_nu_prim_shw2 = cutThetaAngle(df_nu_prim_shw2, thetaAngle = angle) 
    df_cos_prim_shw2 = cutThetaAngle(df_cos_prim_shw2, thetaAngle = angle) 
#df_cos = cutThetaAngle(df_cos, thetaAngle = angle) 

In [ ]:
df_hnl_shw2 = merge_df_prim(df_hnl_shw2, df_hnl_prim_shw2)

df_nu_shw2 = merge_df_prim(df_nu_shw2, df_nu_prim_shw2)

df_cos_shw2 = merge_df_prim(df_cos_shw2, df_cos_prim_shw2)

In [ ]:
df_hnl = apply_shower_cut(df_hnl, df_hnl_shw1, df_hnl_shw2)

df_nu = apply_shower_cut(df_nu, df_nu_shw1, df_nu_shw2)

df_cos = apply_shower_cut(df_cos, df_cos_shw1, df_cos_shw2)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Shower Theta Cut")

print(purity_arr)
print(eff_arr)
print(cut_arr)

<h1> Make Beam Bucket - Post Shower Theta</h1>

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    xmin = 0, xmax = 30, xnbin = 30,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',
                  ifAddLegend = True, addLegend = label
                    )

#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_postshowertheta.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>PFP N Hits</h1>

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_n_hits', 
                    xmin = 0, xmax = 5000, xnbin = 50,
                    xtitle = 'PFP N Hits',
                  ifAddLegend = True, addLegend = label
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"pfp_nhits.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Track Score</h1>

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_track_score', 
                    xmin = 0, xmax = 1, xnbin = 40,
                    xtitle = 'PFP Track Score',
                  ifAddLegend = True, addLegend = label
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"track_score.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Multiplicity</h1>

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_n_stub', 
                    xmin = 0, xmax = 10, xnbin = 10,
                    xtitle = 'n Stubs',
                  ifAddLegend = True, addLegend = label
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nStub.png", dpi = 200)
#-----------------------------------#
plt.show()

#==========================================#
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_n_trks', 
                    xmin = 0, xmax = 10, xnbin = 10,
                    xtitle = 'n Tracks',
                  ifAddLegend = True, addLegend = label
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nTrk.png", dpi = 200)
#-----------------------------------#
plt.show()

#==========================================#
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_n_shws', 
                    xmin = 0, xmax = 10, xnbin = 10,
                    xtitle = 'n Showers',
                  ifAddLegend = True, addLegend = label
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nShw.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
#==========================================#
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_n_trks', 
                    xmin = 0, xmax = 10, xnbin = 10,
                    xtitle = 'n Tracks',
                  ifAddLegend = True, addLegend = label
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nTrk.png", dpi = 200)
#-----------------------------------#
plt.show()

#==========================================#
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_n_shws', 
                    xmin = 0, xmax = 10, xnbin = 10,
                    xtitle = 'n Showers',
                  ifAddLegend = True, addLegend = label
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nShw.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>2 Shower Case: Pi0 Mass </h1>

<h3> Split into 1 shower and multi shower case again! </h3>

In [ ]:
df_cos

In [ ]:
df_hnl_shw1, df_hnl_shw2 = split_my_df(df_hnl)

df_nu_shw1, df_nu_shw2 = split_my_df(df_nu)

#df_cos_shw1, df_cos_shw2 = split_my_df(df_cos)

In [ ]:
df_hnl_shw2['pi0_mass'] = df_hnl_shw2.apply(lambda row: pi0mass(row), axis = 1)

df_nu_shw2['pi0_mass'] = df_nu_shw2.apply(lambda row: pi0mass(row), axis = 1)

df_cos['pi0_mass']  = ""
#df_cos_shw2['pi0_mass'] = df_cos_shw2.apply(lambda row: pi0mass(row), axis = 1)


In [ ]:
df_hnl_prim_shw2 = get_primary_shw_df(df_hnl_shw2)

df_nu_prim_shw2 = get_primary_shw_df(df_nu_shw2)

#df_cos_prim_shw2 = get_primary_shw_df(df_cos_shw2)
#df_cos_prim_shw2 = get_primary_shw_df(df_cos)

In [ ]:
plot_slc_var(df_hnl_prim_shw2, df_nu_prim_shw2, df_cos,
#plot_slc_var(df_hnl_prim_shw2, df_nu_prim_shw2, df_cos_prim_shw2,
                    true_counts, start_counts, 
                    'pi0_mass', 
                    xmin = 0, xmax = 600, xnbin = 30,
                    xtitle = 'Pi 0 Mass [MeV]',
                  ifAddLegend = True, addLegend = label
                    )
# only one line may be specified; full height
plt.axvline(x = 135, color = 'r', linewidth = 1.5, ls = '--')

#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"pizero_mass_precut.png", dpi = 200)
#-----------------------------------#

plt.show()

<h1>Beam Bucket Final Distribution</h1>

In [ ]:
hist, bins = plot_slc_var(
                    df_hnl
                    , df_nu
                    , df_cos
                    , true_counts, start_counts, 
                    'mod_t', 
                    xmin = 0, xmax = 30, xnbin = 30,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',
                  ifAddLegend = True, addLegend = label
                    )

#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_final.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Plot Purity/Efficiency </h1>

In [ ]:
print(eff_arr)
print(purity_arr)
print(cut_arr)

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(10, 6), sharex = True)
ax2 = ax1.twinx()

t = np.arange(0, len(cut_arr) , 1)

effcol = col_dict['MintGreen']
pcol = col_dict['Flamingo']

ax1.plot(t, eff_arr, c = effcol, marker = 'o', markersize = 5, ls='--')
ax2.plot(t, purity_arr, c = pcol, marker = 'o', markersize = 5, ls='--')

ax1.set_ylim(-5,105)
ax2.set_ylim(-5,105)

ax1.grid(color='gainsboro', which='both', axis='both', linestyle='--')

ax1.set_ylabel("Selection Effciency [%]", c = effcol, fontsize = 16)
ax2.set_ylabel("Purity [%]", c = pcol, fontsize = 16)

plt.xticks(range(0, len(t)))
ax1.set_xticklabels(cut_arr, rotation = 45, ha="right", fontsize = 16)

ax1.tick_params(bottom = True, top = True, left = True, right = False)
ax2.tick_params(bottom = True, top = True, left = False, right = True)

ax1.tick_params(axis = 'x', labelsize = 16, direction = 'in')
ax1.tick_params(axis = 'y', labelsize = 16, direction = 'in')
ax2.tick_params(axis = 'y', labelsize = 16, direction = 'in')

ax1.set_title(label, y=1.0, pad=-55, fontsize = 16)
fig.tight_layout()

#-----------------------------------#
if ifSave:
    plt.savefig(savePath+"peff.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Save Only Primary Showers </h1>

In [ ]:
df_hnl = get_primary_shw_df(df_hnl)

df_nu = get_primary_shw_df(df_nu)

df_cos = get_primary_shw_df(df_cos)
#df_cos_prim_shw2 = get_primary_shw_df(df_cos)

In [ ]:
df_hnl[df_hnl['slc_true_event_type'] == 9]

In [ ]:
df_nu[df_nu['slc_true_event_type'] == 9]

In [ ]:
df_cos

<h1> Save Cut Dataframe to Pickle</h1>

In [ ]:
df_hnl.to_pickle("../../pkl_files/v3_April2024/df_postselect_m"+str(m)+"_v3_hnl.pkl", protocol = 5)

In [ ]:
df_nu.to_pickle("../../pkl_files/v3_April2024/df_postselect_m"+str(m)+"_v3_nu.pkl", protocol = 5)

In [ ]:
df_cos.to_pickle("../../pkl_files/v3_April2024/df_postselect_m"+str(m)+"_v3_cos.pkl", protocol = 5)